<a href="https://colab.research.google.com/github/alicewoo0925/ECG-Apnoea-Detection/blob/main/2ndmodel/Second_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference: https://colab.research.google.com/drive/16w3TDn_tAku17mum98EWTmjaLHAJcsk0?usp=sharing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE

# Import Dataset from csv file created in MATLAB

[import reference of the UCI dataset]

In [ ]:
feature_cols = ["RRset_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI"] #name of the features
result_cols = ["A","N"] #classification results of the training set #used one-hot coding
cols = ["RRset_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI","A"] #name of the features

In [ ]:
# training set
X_train_df = pd.read_csv("X_train.csv", names=feature_cols)
T_train_df = pd.read_csv("T_train.csv", names=result_cols)

# concatenate features and target
train_df = pd.concat([X_train_df,T_train_df], axis=1)
del train_df['N'] # drop the second element in target (e.g. [0 1] to 0 and [1 0] to 1)
train_df.head()

,RRset_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI,A
0,1.03070,-3.2674,-3.4358,0.031926,5.5556,0
1,0.93625,-1.5161,-1.4311,0.237140,3.8462,0
2,0.99066,-2.1771,-2.3040,0.099071,5.5556,0
3,1.03140,-2.7058,-2.8820,0.055551,4.5455,0
4,0.95794,-1.7164,-1.8649,0.153660,7.1429,0


In [ ]:
# testing set
feature_cols = ["RRset_mean","log(RRseg_std)","log(SDSD)","RMSSD","RRseg_triI"] #name of the features

X_test_df = pd.read_csv("X_test.csv", names=feature_cols)
X_test_df.head()

,RRset_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI
0,1.03770,-1.9308,-1.8459,0.15652,7.1429
1,1.04550,-1.7437,-1.5828,0.20362,5.5556
2,0.94048,-1.5096,-1.5136,0.21835,16.6670
3,0.98295,-1.5273,-1.5465,0.21120,10.0000
4,0.90269,-1.3942,-1.4470,0.23348,10.0000


# Data cleaning

In [ ]:
train_df[train_df.isna().any(axis=1)]

,RRset_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI,A


In [ ]:
#X_test_df.fillna(0)
X_test_df.replace(np.nan, 0, inplace = True)
X_test_df.replace([np.inf, -np.inf], 0, inplace = True)

In [ ]:
X_test_df[X_test_df.isna().any(axis=1)]

,RRset_mean,log(RRseg_std),log(SDSD),RMSSD,RRseg_triI


# Train,test datasets from the available data

In [ ]:
X_train = train_df[train_df.columns[:-1]].values
Y_train = train_df[train_df.columns[-1]].values

# rescale
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

# smote
smote = SMOTE(k_neighbors=8, random_state=42)
X_train, Y_train = smote.fit_resample(X_train, Y_train)

data_train = np.hstack((X_train, np.reshape(Y_train, (-1, 1))))

In [ ]:
X_test = X_test_df[X_test_df.columns[:]].values

# rescale
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)
#data_train = np.hstack((X_train, np.reshape(Y_train, (-1, 1))))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_model = RandomForestClassifier(n_estimators=1000,min_samples_split=2,min_samples_leaf=1,max_depth=45)
rf_model = rf_model.fit(X_train, Y_train)

In [ ]:
pred = rf_model.predict(X_test)

In [ ]:
pred

array([1, 0, 1, ..., 1, 1, 0])

In [ ]:
proba = rf_model.predict_proba(X_test)

In [ ]:
proba

array([[0.407, 0.593],
       [0.682, 0.318],
       [0.226, 0.774],
       ...,
       [0.402, 0.598],
       [0.356, 0.644],
       [0.926, 0.074]])

In [ ]:
class_df = pd.DataFrame({'Class':np.transpose(pred)})

In [ ]:
class_df

,Class
0,1
1,0
2,1
3,1
4,1
...,...
23177,0
23178,0
23179,1
23180,1


In [ ]:
prob_df = pd.DataFrame(proba,columns=['A','N'])

In [ ]:
prob_df

,A,N
0,0.407000,0.593000
1,0.682000,0.318000
2,0.226000,0.774000
3,0.313125,0.686875
4,0.224000,0.776000
...,...,...
23177,0.944000,0.056000
23178,0.751000,0.249000
23179,0.402000,0.598000
23180,0.356000,0.644000


In [ ]:
result = pd.concat([class_df, prob_df], axis=1)

In [ ]:
result

,Class,A,N
0,1,0.407000,0.593000
1,0,0.682000,0.318000
2,1,0.226000,0.774000
3,1,0.313125,0.686875
4,1,0.224000,0.776000
...,...,...,...
23177,0,0.944000,0.056000
23178,0,0.751000,0.249000
23179,1,0.402000,0.598000
23180,1,0.356000,0.644000


In [ ]:
result.to_csv('result.csv',index=False)